In [ ]:
# Add necessary imports
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from skimage import io
import pandas as pd
import os
import numpy as np

# Flatten the images and masks to use them in Random Forest
def flatten_image(img):
    return img.reshape(-1, img.shape[-1])

def flatten_mask(mask):
    return mask.reshape(-1)

def prepare_data(image_dir, mask_dir, image_files):
    X = []
    y = []
    for img_file in image_files:
        img_path = os.path.join(image_dir, img_file)
        mask_path = os.path.join(mask_dir, img_file.replace('.jpg', '.png'))

        img = io.imread(img_path)
        mask = io.imread(mask_path)

        X.append(flatten_image(img))
        y.append(flatten_mask(mask))

    X = np.vstack(X)
    y = np.hstack(y)

    return X, y

image_files = os.listdir("leaf_disease_segmentation/train_images")
X, y = prepare_data("leaf_disease_segmentation/train_images", "leaf_disease_segmentation/train_masks", image_files)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)

# Evaluate the model
train_accuracy = rf_model.score(X_train, y_train)
val_accuracy = rf_model.score(X_val, y_val)

print(f"Train accuracy: {train_accuracy:.4f}")
print(f"Validation accuracy: {val_accuracy:.4f}")
